In [118]:
#Import pandas for reading and analyzing CSV data:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#read edit ufo
ufo = pd.read_csv('editufo.csv')

#drop stuff so weka doesnt explode
ufo.drop(ufo.loc[ufo['shape'] == 'Missing'].index, inplace=True)
ufo = ufo.replace('\'','', regex=True)

#time stuff
ufo.dropna(subset=['date_time'])
ufo['date_time'] = pd.to_datetime(ufo['date_time'])
ufo['year'] = ufo['date_time'].dt.year
ufo['month'] = ufo['date_time'].dt.month

#drop missing values
ufo = ufo.drop(columns = ['text'])

In [119]:
from datetime import datetime
#sunset
sunset = pd.read_csv('averagedaytime.csv')
#sunset['Month'] = pd.to_datetime(sunset['Month'])
sunset['Sunrise'] = pd.to_datetime(sunset['Sunrise']).dt.time
sunset['Sunset'] = pd.to_datetime(sunset['Sunset']).dt.time
sunset['index'] = (sunset.index + 1)
sunset.head()

,Month,Sunrise,Sunset,index
0,January,07:23:00,17:11:00,1
1,February,06:57:00,17:46:00,2
2,March,07:17:00,19:16:00,3
3,April,06:29:00,19:46:00,4
4,May,05:53:00,20:15:00,5


In [120]:
ufo['time']= ufo['date_time'].dt.time
sunset['month']=sunset['index']
sunset.head()
temp = ufo.merge(sunset,on=['month'],how='outer')
temp.head()

,city,state,date_time,shape,city_latitude,city_longitude,duration_sec,year,month,time,Month,Sunrise,Sunset,index
0,Chester,VA,2019-12-12 18:43:00,light,37.343152,-77.408582,5.0,2019,12,18:43:00,December,07:18:00,16:48:00,12
1,Philadelphia,PA,2004-12-24 00:00:00,disk,39.990635,-75.145801,120.0,2004,12,00:00:00,December,07:18:00,16:48:00,12
2,Malibu,CA,2004-12-28 14:00:00,triangle,34.068169,-118.816399,60.0,2004,12,14:00:00,December,07:18:00,16:48:00,12
3,Ypsilanti,MI,2005-12-05 07:00:00,other,42.236335,-83.602970,600.0,2005,12,07:00:00,December,07:18:00,16:48:00,12
4,Smithshire,IL,2005-12-05 20:00:00,other,40.749200,-90.771300,60.0,2005,12,20:00:00,December,07:18:00,16:48:00,12


In [121]:
temp['day'] = ((temp['Sunrise']<temp['time']) & (temp['time']<temp['Sunset']))
temp = temp.drop(columns = ['Month','Sunrise','time','Sunset','index'])
temp = temp.drop(columns = ['date_time'])
temp = temp.replace(to_replace = False, value = 'Night')
temp = temp.replace(to_replace = True, value = 'Day')
temp.head()


,city,state,shape,city_latitude,city_longitude,duration_sec,year,month,day
0,Chester,VA,light,37.343152,-77.408582,5.0,2019,12,Night
1,Philadelphia,PA,disk,39.990635,-75.145801,120.0,2004,12,Night
2,Malibu,CA,triangle,34.068169,-118.816399,60.0,2004,12,Day
3,Ypsilanti,MI,other,42.236335,-83.602970,600.0,2005,12,Night
4,Smithshire,IL,other,40.749200,-90.771300,60.0,2005,12,Night


In [123]:
#save as CSV
temp.to_csv("simple_ufo.csv",index=False)

In [126]:
##regional lists

Northeast = 'CT,DE,ME,MD,MA,NH,NJ,NY,PA,RI,VT'
Midwest = 'IL,IN,IA,KS,MI,MN,MO,NE,ND,OH,SD,WI'
South = 'AL,AR,FL,GA,KY,LA,MS,NC,SC,TN,VA,WV,TX'
West = 'AK,AZ,CA,CO,HI,ID,MT,NV,NM,OR,UT,WA,WY'

def makeList(states):
    state_list = []
    stateSplit = states.split(',')
    for i in stateSplit:
        state_list.append(i)
    return state_list

NE_states = makeList(Northeast)
MW_states = makeList(Midwest)
S_states = makeList(South)
W_states = makeList(West)


NE_ufo = temp[temp['state'].isin(NE_states)]
MW_ufo = temp[temp['state'].isin(MW_states)]
S_ufo = temp[temp['state'].isin(S_states)]
W_ufo = temp[temp['state'].isin(W_states)]

NE_ufo.head()

##Save as CSV, dataset split up regionally
NE_ufo.to_csv("NE_ufo.csv", index=False)
MW_ufo.to_csv("MW_ufo.csv", index=False)
S_ufo.to_csv("S_ufo.csv", index=False)
W_ufo.to_csv("W_ufo.csv", index=False)
